In [0]:
%pip install --upgrade databricks-sdk
dbutils.library.restartPython()

In [0]:
import databricks.sdk
print(databricks.sdk.version.__version__)


0.18.0


In [0]:
# ワークスペース用クライアントを作成、クラスター再起動のたびに実行してください

from databricks.sdk import WorkspaceClient

w = WorkspaceClient(host="", token="")


In [0]:
from databricks.sdk import AccountClient

a = AccountClient(
  host                = "https://accounts.azuredatabricks.net",
  account_id          = "",
  client_id     = "",
  client_secret = "",
  auth_type           = ""
)


In [0]:
import os, time, pprint
from databricks.sdk.service import compute
from databricks.sdk.service.compute import ClusterAccessControlRequest

In [0]:
# クラスター作成
def create_cluster(cluster_name,user_name,node = 'Standard_DS3_v2'):

  latest = w.clusters.select_spark_version(latest=True,ml=True)


  clstr = w.clusters.create(cluster_name=cluster_name,
                            spark_version=latest,
                            autotermination_minutes=15,
                            single_user_name= user_name,
                            spark_conf={
                              "spark.databricks.cluster.profile": "singleNode",
                              "spark.master": "local[*, 4]"
                            },
                            node_type_id=node,
                            custom_tags={
                              "ResourceClass": "SingleNode",
                              "Type": "Personal"
                            },
                            policy_id="0018B70A5594EB56",
                            data_security_mode=compute.DataSecurityMode.SINGLE_USER
                            )
  w.clusters.set_permissions(cluster_id=clstr.cluster_id,access_control_list=[ClusterAccessControlRequest(user_name=user_name,permission_level=compute.ClusterPermissionLevel.CAN_RESTART)])
  


In [0]:
#0207 払い出し除外用

usrid_list = ['']

In [0]:
for u in usrid_list:
    try:
        clst_n = u + " 's cluster"
        usr_n = u
        create_cluster(cluster_name = clst_n, user_name = usr_n)
    except(Exception) as e:
        print(u, ' does not exist')